In [0]:
# -*- coding: utf-8 -*-
import dataiku
import os
from markitdown import MarkItDown
import tempfile
import openai
from openai import OpenAI
from unstructured.partition.pdf import partition_pdf
from PIL import Image

In [0]:
openai_client = OpenAI(api_key="xxx")
md = MarkItDown(llm_client=openai_client, llm_model="gpt-4o")

client = dataiku.api_client()
project = client.get_default_project()
llm_list = project.list_llms()
for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")
llm = project.get_llm("openai:OpenAI-FA:gpt-4o-mini")

In [0]:
# Folders
A220_tech_docs = dataiku.Folder("SoQWOnhR")          # Input folder
A220_tech_docs_prep = dataiku.Folder("AXB1Cyno")    # Output folder

# Lister les fichiers PDF
pdf_files = [f for f in A220_tech_docs.list_paths_in_partition() if f.lower().endswith(".pdf") ]
pdf_files = [f for f in A220_tech_docs.list_paths_in_partition() if f.lower().endswith(".pdf") and "hydro" in f.lower()]
print(pdf_files)

In [0]:
prompt_image = "Describe shortly this image given the context of the pdf."


def prompt_image_with_context(summary, image_context):
    return (
        "The aim is to describe an image extracted from a pdf document."
        "I will first provide you a short summary of a document"
        "Then the text then a context for the image"
        "Before you execute the description"
        "\n"
        "# Summary"
        f"{summary}"
        "\n"
        "# Image context"
        f"{image_context}"
        "\n"
        "# Execute the description\n"
        f"{prompt_image}\n"
    )

def comment_image_with_llm(image_path):
    with open(image_path, "rb") as img_file:
        response = openai.Image.create(file=img_file, prompt=promt_image)
    return response.get("description", "No description.")

In [0]:
def pdf_2_md(file):
    # Convertir en Markdown
    md_content = md.convert(temp_pdf.name)
    
    # Afficher le nombre de lignes dans le contenu Markdown
    num_lines = len(md_content.text_content.splitlines())
    print(f"Nombre de lignes : {num_lines}")
    
    return md_content.text_content



def pdf_2_md_with_images(pdf_path):
    # Extraire le contenu structuré
    elements = partition_pdf(filename=pdf_path)
    markdown_output = ""

    # Parcourir les éléments extraits
    for element in elements:
        if element.type == "Text":
            markdown_output += f"{element.text}\n\n"
        elif element.type == "Image":
            # Sauvegarder l'image
            image_path = f"{element.id}.png"
            with A220_tech_docs_prep.get_writer(image_path) as writer:
                writer.write(element.content)

            # Ajouter une référence Markdown à l'image
            markdown_output += f"![Image](./{os.path.basename(image_path)})\n"

            # Ajouter un commentaire généré par le LLM
            comment = comment_image_with_llm(image_path)
            markdown_output += f"> _Commentaire sur l'image : {comment}_\n\n"

    return markdown_output

prompt_summarization = "Summarize the content of the following file content in one small paragraph:\n"

def summarize_with_llm(md_text):
    # Create and run a completion query
    completion = llm.new_completion()
    prompt = (
        f"{prompt_summarization}\n"
        f"# File Content\n"
        f"{md_text}"
    )
    completion.with_message(prompt[:10000])
    resp = completion.execute()

    # Display the LLM output
    if resp.success:
        return resp.text
    else:
        print(dir(resp))
        return resp.text

In [0]:
for pdf_file in pdf_files:
    # Lire le contenu PDF
    with A220_tech_docs.get_download_stream(pdf_file) as f:
        pdf_data = f.read()

        # Utiliser un fichier temporaire pour la conversion
        with tempfile.NamedTemporaryFile(delete=True, suffix=".pdf") as temp_pdf:
            temp_pdf.write(pdf_data)
            temp_pdf.flush()  # Assurez-vous que le contenu est écrit sur le disque

            #md_text = pdf_2_md(temp_pdf.name)
            
            md_text = pdf_2_md_with_images(temp_pdf.name)
            
            # Écrire le fichier .md
            md_file_name = os.path.splitext(pdf_file)[0] + ".md"
            with A220_tech_docs_prep.get_writer(md_file_name) as writer:
                writer.write(md_text.encode('utf-8'))

print(llm)
print(summarize_with_llm(md_text))